In [2]:
# !pip install bioc

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive
# %cd "My Drive"
# %cd "Colab Notebooks"
# # /content/drive/My Drive/Colab Notebooks
# %cd "Dr.Kojima_JOB"

Mounted at /content/drive
/content/drive
/content/drive/My Drive
/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/Dr.Kojima_JOB


# BioCCollectionオブジェクト調査

In [38]:
import bioc

# bioc.bioc.BioCCollection型
with open("data_original/aimed_bioc.xml", 'r') as fp:
    anem_bioc = bioc.load(fp)

# # bioc.biocxml.decoder.BioCXMLDocumentReader型
# reader = bioc.BioCXMLDocumentReader("data/anem_bioc.xml")

# # bioc.bioc.BioCDocument型
# for document in reader:
#   print(document)
#   print(type(document))

In [13]:
for key, value in anem_bioc.__dict__.items():
    print(key, ':',type(value))
#     print(key, ':', value,type(value))

infons : <class 'dict'>
encoding : <class 'str'>
version : <class 'str'>
standalone : <class 'bool'>
source : <class 'str'>
date : <class 'str'>
key : <class 'str'>
documents : <class 'list'>


In [14]:
# anem_bioc.__str__()

In [15]:
document = anem_bioc.__dict__["documents"]

In [16]:
for key, value in document[0].__dict__.items():
    print(key, ':',type(value))
#     print(key, ':', value,type(value))

infons : <class 'dict'>
annotations : <class 'list'>
relations : <class 'list'>
id : <class 'str'>
passages : <class 'list'>


In [19]:
type(document[0].__dict__["passages"][0])

bioc.bioc.BioCPassage

In [20]:
a_BioCPassage_list = document[0].__dict__["passages"]

In [21]:
for key, value in a_BioCPassage_list[0].__dict__.items():
  print(key, ':', value,type(value))

infons : {} <class 'dict'>
annotations : [BioCAnnotation[id=T1,text='delta-catenin',infons=[file=ann,type=protein],locations=[BioCLocation[offset=19,length=13]],], BioCAnnotation[id=T3,text='presenilin (PS) 1',infons=[file=ann,type=protein],locations=[BioCLocation[offset=122,length=17]],], BioCAnnotation[id=T2,text='presenilin 1',infons=[file=ann,type=protein],locations=[BioCLocation[offset=66,length=12]],], BioCAnnotation[id=T6,text='PS1',infons=[file=ann,type=protein],locations=[BioCLocation[offset=415,length=3]],], BioCAnnotation[id=T7,text='PS-2',infons=[file=ann,type=protein],locations=[BioCLocation[offset=441,length=4]],], BioCAnnotation[id=T4,text='delta-catenin',infons=[file=ann,type=protein],locations=[BioCLocation[offset=182,length=13]],], BioCAnnotation[id=T5,text='delta-catenin',infons=[file=ann,type=protein],locations=[BioCLocation[offset=313,length=13]],], BioCAnnotation[id=T10,text='PS1',infons=[file=ann,type=protein],locations=[BioCLocation[offset=513,length=3]],], BioC

In [39]:
a_BioCAnnotation_list = a_BioCPassage_list[0].__dict__["annotations"]
a_BioCRelations_list = a_BioCPassage_list[0].__dict__["relations"]

In [40]:
for key, value in a_BioCAnnotation_list[0].__dict__.items():
  print(key, ':', value,type(value))

infons : {'file': 'ann', 'type': 'protein'} <class 'dict'>
locations : [BioCLocation[offset=19,length=13]] <class 'list'>
id : T1 <class 'str'>
text : delta-catenin <class 'str'>


In [41]:
for key, value in a_BioCRelations_list[0].__dict__.items():
  print(key, ':', value,type(value))

infons : {'relation type': 'Interaction', 'file': 'ann', 'type': 'Relation'} <class 'dict'>
id : R4 <class 'str'>
nodes : [BioCNode[refid=T12,role=Arg1], BioCNode[refid=T13,role=Arg2]] <class 'list'>


In [42]:
a_BioCRelations_Nodes = a_BioCRelations_list[0].__dict__["nodes"]

In [ ]:
Nodes_set = set()
for BioCNode in a_BioCRelations_Nodes:
    print(BioCNode.__dict__["refid"])
    Nodes_set.add(BioCNode.__dict__["refid"])
    
Nodes_set

# データ作成

In [6]:
import os
import bioc
import itertools
import csv

In [4]:
def get_PairData(filename):
    # bioc.biocxml.decoder.BioCXMLDocumentReader型
    all_data = []
    reader = bioc.BioCXMLDocumentReader("data_original/"+filename+".xml")

    # bioc.bioc.BioCDocument型
    for document in reader:
        document_id = document.__dict__["id"]

        for passage in document.__dict__["passages"]:
            text = passage.__dict__["text"]

            ann_ids_texts = []
            for annotation in passage.__dict__["annotations"]:
                ann_id = annotation.__dict__["id"]
                ann_text = annotation.__dict__["text"]
                ann_ids_texts.append([ann_id,ann_text])

            for pair in itertools.combinations(ann_ids_texts, 2):
                a,b = list(pair)

                row = [document_id,text,a[0],b[0],a[1],b[1],set([a[0],b[0]])]
                all_data.append(row)
                
    return all_data

In [5]:
def get_RelDataDict(filename):
    # bioc.biocxml.decoder.BioCXMLDocumentReader型
    all_rel_data_dict = dict()
    reader = bioc.BioCXMLDocumentReader("data_original/"+filename+".xml")
    
    for document in reader:
        document_id = document.__dict__["id"]
        document_rels = []

        for passage in document.__dict__["passages"]:

            for relations in passage.__dict__["relations"]:
                a_node = set()
                for nodes in relations.__dict__["nodes"]:
                    a_node.add(nodes.__dict__["refid"])
                document_rels.append(a_node)

        all_rel_data_dict[document_id] = document_rels
    return all_rel_data_dict

In [6]:
def Create_CSV(filename):
    PairData = get_PairData(filename)
    RelDataDict = get_RelDataDict(filename)
    
    newPairData = []
    for pair in PairData:
        TorF = "FALSE"
        if pair[-1] in RelDataDict[pair[0]]:
            TorF = "TRUE"
        newPairData.append(pair[:-1]+[TorF])
        
    with open("data/"+filename+".csv", "w") as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(newPairData)

In [7]:
files = os.listdir("data_original")

val_name = []
for a_file in files:
  print(a_file)
  with open("data_original/"+a_file, 'r') as fp:
    dynmc_name = a_file.replace(".xml","")
    locals()[dynmc_name] = bioc.load(fp)
    val_name.append(dynmc_name)

aimed_bioc.xml
bioinfer_bioc.xml
hprd50_bioc.xml
iepa_bioc.xml
lll_bioc.xml


In [6]:
for filename in val_name:
    print(filename)
    Create_CSV(filename)

aimed_bioc
bioinfer_bioc
hprd50_bioc
iepa_bioc
lll_bioc


# データ概要の集計

In [9]:
import collections
import os
import bioc
import csv

In [15]:
def get_Summary(filename):
    with open('data/'+filename+'.csv') as rf:
        reader = csv.reader(rf)
        data = [row for row in reader]
    
    #ラベルごとの辞書作成
    label_dict = dict()
    key_l = [row[2] for row in data]+[row[3] for row in data]
    ent_1 = [row[4] for row in data]+[row[5] for row in data]
    
    for key in set(key_l):
        label_dict[key] = set()

    for i in range(len(key_l)):
        key = key_l[i]
        ent = ent_1[i]
        label_dict[key].add(ent)
    
    #TorFの件数と比率
    t = len([row[-1] for row in data if row[-1] == "TRUE"])
    f = len(data)-t
    t_percent = t/len(data)*100
    f_percent = f/len(data)*100
    
    wf = open('data_summary/'+filename+'T_F.txt','w',encoding='UTF-8',newline='\n')
    wf.write("【Ratio of T to F】\n")
    wf.write("TRUE:FALES = "+str(t)+" : "+str(f)+"\n")
    wf.write(str(t_percent)+"% : "+str(f_percent)+"%\n")
    wf.write("\n")
    wf.write("\n")
    wf.write("【Proteins for each label】\n")
    for key,value in label_dict.items():
        wf.write(str(key)+":"+str(value)+"\n")
    wf.close()

In [25]:
def get_TF_Summary(filename):
    with open('data/'+filename+'.csv') as rf:
        reader = csv.reader(rf)
        data = [row for row in reader]
    
    #TorFの件数と比率
    t = len([row[-1] for row in data if row[-1] == "TRUE"])
    f = len(data)-t
    t_percent = t/len(data)*100
    f_percent = f/len(data)*100
    
    wf = open('data_summary/TF/'+filename+'_TF.txt','w',encoding='UTF-8',newline='\n')
    wf.write("【Ratio of T to F】\n")
    wf.write("TRUE:FALES = "+str(t)+" : "+str(f)+"\n")
    wf.write(str(t_percent)+"% : "+str(f_percent)+"%\n")
    wf.close()

In [26]:
def get_Prot_Summary(filename):
    with open('data/'+filename+'.csv') as rf:
        reader = csv.reader(rf)
        data = [row for row in reader]
    
    #プロテイン名ごとの辞書作成
    prot_1 = [row[4] for row in data]+[row[5] for row in data]
    prot_set = sorted(list(set(prot_1)))
    prot_dict = collections.Counter(prot_1)
        
    with open('data_summary/PROT/'+filename+'_prot.csv','w',encoding='UTF-8',newline='\n') as f:
        writer = csv.writer(f)
        for prot in prot_set:
            writer.writerow([prot,prot_dict[prot]])

In [27]:
files = os.listdir("data_original")

val_name = []
for a_file in files:
    with open("data_original/"+a_file, 'r') as fp:
        dynmc_name = a_file.replace(".xml","")
        locals()[dynmc_name] = bioc.load(fp)
        val_name.append(dynmc_name)

In [28]:
for filename in val_name:
    print(filename)
    get_TF_Summary(filename)

aimed_bioc
bioinfer_bioc
hprd50_bioc
iepa_bioc
lll_bioc


In [29]:
for filename in val_name:
    print(filename)
    get_Prot_Summary(filename)

aimed_bioc
bioinfer_bioc
hprd50_bioc
iepa_bioc
lll_bioc
